In [1]:
import random    
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import timeit

특정 비트수를 가지는 이진수로 변환해주기 위한 함수

In [2]:
def trans(gene, num):
    T= "0123456789ABCDEF"
    q, mod = divmod(gene, num) # 몫과 나머지를 return 해줌
    if q == 0:
        return T[mod]
    else:
        return trans(q, num) + T[mod]

In [3]:
def convert(gene, num, bit):
    gene = str(trans(gene, num))
    length = len(gene)
    if(length < bit):
        for i in range(bit - length):
            gene = "0" + gene
    return gene

초기에 필요한 값들 정의

In [4]:
def init():    
    global learning_rate_group
    x = 0.0999 / 255
    learning_rate_group = [0.0001 + x * i for i in range(256)]    
    global learning_rate_dic
    learning_rate_dic = {}
    for i in range(len(learning_rate_group)):
        learning_rate_dic[learning_rate_group[i]] = convert(i, 2, 8)

    
    global drop_out_group
    x = 0.5 / 15
    drop_out_group = [0.5 + x * i for i in range(16)]
    global drop_out_dic
    drop_out_dic = {}
    for i in range(len(drop_out_group)):
        drop_out_dic[drop_out_group[i]] = convert(i, 2, 4)

    
    global batch_size_group
    batch_size_group = [50, 100, 200, 250, 400, 500, 1000, 1250]
    global batch_size_dic
    batch_size_dic = {}
    for i in range(len(batch_size_group)):
        batch_size_dic[batch_size_group[i]] = convert(i, 2, 3)

    
    global layer_group
    layer_group = [1, 2, 3, 4]
    global layer_dic
    layer_dic = {}
    for i in range(len(layer_group)):
        layer_dic[layer_group[i]] = convert(i, 2, 2)


처음 유전자의 값을 랜덤으로 설정

In [5]:
def Initialize():
    gene = []
    
    learning_rate = random.choice(learning_rate_group)
    gene.append(learning_rate)
    
    drop_out_1 = random.choice(drop_out_group)
    drop_out_2 = random.choice(drop_out_group)
    gene.append(drop_out_1)
    gene.append(drop_out_2)
    
    batch_size = random.choice(batch_size_group)
    gene.append(batch_size)
    
    layer = random.choice(layer_group)
    
    gene.append(layer)
    
    return gene

2진수로 Encoding하는 부분

In [6]:
#learning rate : 0.5~0 --> 8bits
def Encoding(gene): #내가 임의로 선택한 숫자를 준다(이진화 전)
    learning_rate = gene[0]
    drop_out_1 = gene[1]
    drop_out_2 = gene[2]
    batch_size = gene[3]
    layer = gene[4]
    
    gene = []

    learning_rate = learning_rate_dic[learning_rate]
    for i in range(len(learning_rate)):
        gene.append(int(learning_rate[i]))
    
    
    drop_out_1 = drop_out_dic[drop_out_1]
    drop_out_1 = str(drop_out_1)
    for i in range(len(drop_out_1)):
        gene.append(int(drop_out_1[i]))
    drop_out_2 = drop_out_dic[drop_out_2]
    drop_out_2 = str(drop_out_2)
    for i in range(len(drop_out_2)):
        gene.append(int(drop_out_2[i]))
        
    batch_size = batch_size_dic[batch_size]
    batch_size = str(batch_size)
    for i in range(len(batch_size)):
        gene.append(int(batch_size[i]))

    layer = layer_dic[layer]
    layer = str(layer)
    for i in range(len(layer)):
        gene.append(int(layer[i]))
    
    return gene
    # 이진화지만 이런 형태로 나감 [1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0]



Evaluation 하기 위해 CNN을 구성하는 부분과 Evaluation 함수

In [7]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))


def model_1(X, w1, w2, w3, w4, b1, b2, b3, b4,w_f1,w_f2,w_f3,w_f4, w_o, p_keep_conv, p_keep_hidden):
    l1a = tf.nn.relu(tf.nn.conv2d(X, w1,                       # l1a shape=(?, 28, 28, 32)
                        strides=[1, 1, 1, 1], padding='SAME')+b1)
    l1 = tf.nn.max_pool(l1a, ksize=[1, 2, 2, 1],              # l1 shape=(?, 14, 14, 32)
                        strides=[1, 2, 2, 1], padding='SAME')
    l1 = tf.reshape(l1, [-1, w_f1.get_shape().as_list()[0]])
    l1 = tf.nn.dropout(l1, p_keep_conv)

    lf1 = tf.nn.relu(tf.matmul(l1, w_f1))
    lf1 = tf.nn.dropout(lf1, p_keep_hidden)

    pyx = tf.matmul(lf1, w_o)
    return pyx

def model_2(X, w1, w2, w3, w4, b1, b2, b3, b4,w_f1,w_f2,w_f3,w_f4, w_o, p_keep_conv, p_keep_hidden):
    l1a = tf.nn.relu(tf.nn.conv2d(X, w1,                       # l1a shape=(?, 28, 28, 32)
                        strides=[1, 1, 1, 1], padding='SAME')+b1)
    l1 = tf.nn.max_pool(l1a, ksize=[1, 2, 2, 1],              # l1 shape=(?, 14, 14, 32)
                        strides=[1, 2, 2, 1], padding='SAME')
    l1 = tf.nn.dropout(l1, p_keep_conv)

    l2a = tf.nn.relu(tf.nn.conv2d(l1, w2,                     # l2a shape=(?, 14, 14, 64)
                        strides=[1, 1, 1, 1], padding='SAME')+b2)
    l2 = tf.nn.max_pool(l2a, ksize=[1, 2, 2, 1],              # l2 shape=(?, 7, 7, 64)
                        strides=[1, 2, 2, 1], padding='SAME')
    l2 = tf.reshape(l2, [-1, w_f2.get_shape().as_list()[0]])
    l2 = tf.nn.dropout(l2, p_keep_conv)

    lf2 = tf.nn.relu(tf.matmul(l2, w_f2))
    lf2 = tf.nn.dropout(lf2, p_keep_hidden)

    pyx = tf.matmul(lf2, w_o)
    return pyx

def model_3(X, w1, w2, w3, w4, b1, b2, b3, b4,w_f1,w_f2,w_f3,w_f4, w_o, p_keep_conv, p_keep_hidden):
    l1a = tf.nn.relu(tf.nn.conv2d(X, w1,                       # l1a shape=(?, 28, 28, 32)
                        strides=[1, 1, 1, 1], padding='SAME')+b1)
    l1 = tf.nn.max_pool(l1a, ksize=[1, 2, 2, 1],              # l1 shape=(?, 14, 14, 32)
                        strides=[1, 2, 2, 1], padding='SAME')
    l1 = tf.nn.dropout(l1, p_keep_conv)

    l2a = tf.nn.relu(tf.nn.conv2d(l1, w2,                     # l2a shape=(?, 14, 14, 64)
                        strides=[1, 1, 1, 1], padding='SAME')+b2)
    l2 = tf.nn.max_pool(l2a, ksize=[1, 2, 2, 1],              # l2 shape=(?, 7, 7, 64)
                        strides=[1, 2, 2, 1], padding='SAME')
    l2 = tf.nn.dropout(l2, p_keep_conv)

    l3a = tf.nn.relu(tf.nn.conv2d(l2, w3,                     # l3a shape=(?, 7, 7, 128)
                        strides=[1, 1, 1, 1], padding='SAME')+b3)
    l3 = tf.nn.max_pool(l3a, ksize=[1, 2, 2, 1],              # l3 shape=(?, 4, 4, 128)
                        strides=[1, 2, 2, 1], padding='SAME')
    l3 = tf.reshape(l3, [-1, w_f3.get_shape().as_list()[0]])    # reshape to (?, 2048)
    l3 = tf.nn.dropout(l3, p_keep_conv)

    lf3 = tf.nn.relu(tf.matmul(l3, w_f3))
    lf3 = tf.nn.dropout(lf3, p_keep_hidden)

    pyx = tf.matmul(lf3, w_o)
    return pyx

def model_4(X, w1, w2, w3, w4, b1, b2, b3, b4,w_f1,w_f2,w_f3,w_f4, w_o, p_keep_conv, p_keep_hidden):
    l1a = tf.nn.relu(tf.nn.conv2d(X, w1,                       # l1a shape=(?, 28, 28, 32)
                        strides=[1, 1, 1, 1], padding='SAME')+b1)
    l1 = tf.nn.max_pool(l1a, ksize=[1, 2, 2, 1],              # l1 shape=(?, 14, 14, 32)
                        strides=[1, 2, 2, 1], padding='SAME')
    l1 = tf.nn.dropout(l1, p_keep_conv)

    l2a = tf.nn.relu(tf.nn.conv2d(l1, w2,                     # l2a shape=(?, 14, 14, 64)
                        strides=[1, 1, 1, 1], padding='SAME')+b2)
    l2 = tf.nn.max_pool(l2a, ksize=[1, 2, 2, 1],              # l2 shape=(?, 7, 7, 64)
                        strides=[1, 2, 2, 1], padding='SAME')
    l2 = tf.nn.dropout(l2, p_keep_conv)

    l3a = tf.nn.relu(tf.nn.conv2d(l2, w3,                     # l3a shape=(?, 7, 7, 128)
                        strides=[1, 1, 1, 1], padding='SAME')+b3)
    l3 = tf.nn.max_pool(l3a, ksize=[1, 2, 2, 1],              # l3 shape=(?, 4, 4, 128)
                        strides=[1, 2, 2, 1], padding='SAME')
    #l3 = tf.reshape(l3, [-1, w_f3.get_shape().as_list()[0]])    # reshape to (?, 2048)
    l3 = tf.nn.dropout(l3, p_keep_conv)
    
    l4a = tf.nn.relu(tf.nn.conv2d(l3, w4,                     # l3a shape=(?, 7, 7, 128)
                        strides=[1, 1, 1, 1], padding='SAME')+b4)
    l4 = tf.nn.max_pool(l4a, ksize=[1, 2, 2, 1],              # l3 shape=(?, 4, 4, 128)
                        strides=[1, 2, 2, 1], padding='SAME')
    l4 = tf.reshape(l4, [-1, w_f4.get_shape().as_list()[0]])    # reshape to (?, 2048)
    l4 = tf.nn.dropout(l4, p_keep_conv)

    lf4 = tf.nn.relu(tf.matmul(l4, w_f4))
    lf4 = tf.nn.dropout(lf4, p_keep_hidden)

    pyx = tf.matmul(lf4, w_o)
    return pyx


In [8]:
def CNN(layer, batch_size, learning_rate, drop_out_1, drop_out_2):

    #tf.nn.dropout에 들어가는 확률은 각 요소가 유지되는 확률이다..
    mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
    X = tf.placeholder(tf.float32, [None, 784])
    X_img = tf.reshape(X, [-1, 28, 28, 1]) #??X28X28X1 자동 셋팅
    Y = tf.placeholder(tf.float32, [None, 10])
    dropout = tf.placeholder(tf.float32)
    dropout2 = tf.placeholder(tf.float32)


    w1 = init_weights([3, 3, 1, 32])       # 3x3x1 conv, 32 outputs
    b1 = init_weights([32])
    w2 = init_weights([3, 3, 32, 64]) # 3x3x32 conv, 64 outputs
    b2 = init_weights([64])
    w3 = init_weights([3, 3, 64, 128])# 3x3x32 conv, 128 outputs
    b3 = init_weights([128])
    w4 = init_weights([3, 3, 128, 256])
    b4 = init_weights([256])
    w_f1 = init_weights([32 * 14 * 14, 625])
    w_f2 = init_weights([64 * 7 * 7, 625])
    w_f3 = init_weights([128 * 4 * 4, 625]) # FC 128 * 4 * 4 inputs, 625 outputs
    w_f4 = init_weights([256 * 2 * 2, 625])
    w_o = init_weights([625, 10])         # FC 625 inputs, 10 outputs (labels)



    if layer == 1:
        hypothesis = model_1(X_img, w1, w2, w3, w4, b1, b2, b3, b4,w_f1,w_f2,w_f3,w_f4, w_o, dropout, dropout2)

    if layer == 2:
        hypothesis = model_2(X_img, w1, w2, w3, w4, b1, b2, b3, b4,w_f1,w_f2,w_f3,w_f4, w_o, dropout, dropout2)
    
    if layer == 3:
        hypothesis = model_3(X_img, w1, w2, w3, w4, b1, b2, b3, b4,w_f1,w_f2,w_f3,w_f4, w_o, dropout, dropout2)
    
    if layer == 4:
        hypothesis = model_4(X_img, w1, w2, w3, w4, b1, b2, b3, b4,w_f1,w_f2,w_f3,w_f4, w_o, dropout, dropout2)
    

    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = hypothesis,labels=Y))
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.arg_max(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Launch the graph in a session

    with tf.Session() as sess:
        with tf.device("/gpu:0"):
            
    
            tf.initialize_all_variables().run()
    
            for epoch in range(20):
                print(epoch+1, "번째 ")
                total_batch = int(mnist.train.num_examples / batch_size)
                for i in range(total_batch):
                    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
                    sess.run(optimizer, feed_dict={X: batch_xs, Y: batch_ys, dropout : drop_out_1, dropout2 : drop_out_2})
        
            
        
            test_accuracy = accuracy.eval(feed_dict = {X: mnist.test.images, Y:mnist.test.labels, dropout : 1.0 , dropout2 : 1.0})
            print("test accuracy : ", test_accuracy)
            
          
    

    return test_accuracy 

In [9]:
def Evaluation(gene): 
    #loss function을 구하기 위한 설정
    #print("Evaluation 실행")
    
    global learning_rate_dic
    global drop_out_dic
    global batch_size_dic
    global layer_dic
    
    gene_new = []
    learning_rate = []
    learning_rate = gene[:8]
    learning_rate = ''.join(map(str, learning_rate))
    for key, value in learning_rate_dic.items():
        if value == learning_rate:
            learning_rate = key
            break
    gene_new.append(learning_rate)
    #print(learning_rate)
    drop_out_1 = gene[8:12]
    drop_out_1 = ''.join(map(str, drop_out_1))
    for key, value in drop_out_dic.items():
        if value == drop_out_1:
            drop_out_1 = key
            break

    drop_out_2 = gene[12:16]
    drop_out_2 = ''.join(map(str, drop_out_2))
    for key, value in drop_out_dic.items():
        if value == drop_out_2:
            drop_out_2 = key
            break
    
    gene_new.append(drop_out_1)
    gene_new.append(drop_out_2)
    
    batch_size = gene[16:19]
    batch_size = ''.join(map(str, batch_size))
    for key, value in batch_size_dic.items():
        if value == batch_size:
            batch_size = key
            break
    gene_new.append(batch_size)
    
    layer = gene[19:]
    layer = ''.join(map(str, layer))
    for key, value in layer_dic.items():
        if value == layer:
            layer = key
            break
    gene_new.append(layer)
    
    #loss 값 구하기 

    cost_value = CNN(layer, batch_size, learning_rate, drop_out_1, drop_out_2)
    return cost_value
 

selection

In [10]:
def Selection(cost, gene_group): #Goal is to select parents for crossover. 
    #차라리 dic형태로 보내자.. 그게 나을 듯.... 수정 필요
    
    total = 0
    for i in range(len(cost)):
        total += cost[i]
    
   
    p=[cost[0]/total,cost[1]/total,cost[2]/total,cost[3]/total,cost[4]/total,cost[5]/total,cost[6]/total,cost[7]/total, cost[8]/total, cost[9]/total,
       cost[10]/total,cost[11]/total,cost[12]/total,cost[13]/total,cost[14]/total,cost[15]/total,cost[16]/total,cost[17]/total, cost[18]/total, cost[19]/total,
       cost[20]/total,cost[21]/total,cost[22]/total,cost[23]/total,cost[24]/total,cost[25]/total,cost[26]/total,cost[27]/total, cost[28]/total, cost[29]/total,
       cost[30]/total,cost[31]/total,cost[32]/total,cost[33]/total,cost[34]/total,cost[35]/total,cost[36]/total,cost[37]/total, cost[38]/total, cost[39]/total,
       cost[40]/total,cost[41]/total,cost[42]/total,cost[43]/total,cost[44]/total,cost[45]/total,cost[46]/total,cost[47]/total, cost[48]/total, cost[49]/total]
    rnd_num = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
             10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
             20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
             30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
             40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
    
    p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12, p13, p14, p15, p16, p17, p18, p19, p20, p21, p22, p23, p24, p25, p26, p27, p28, p29, p30 = np.random.choice(rnd_num, 30, replace = False, p=p)
    
    
    print("selected_genegene_frop", gene_group[p1])
    '''
    if parent_1 == 1: parent_1 = gene_group[0]
    elif parent_1 == 2: parent_1 = gene_group[1]
    elif parent_1 == 3: parent_1 = gene_group[2]
    elif parent_1 == 4: parent_1 = gene_group[3]
    elif parent_1 == 5: parent_1 = gene_group[4]
        
    
    if parent_2 == 1: parent_2 = gene_group[0]
    elif parent_2 == 2: parent_2 = gene_group[1]
    elif parent_2 == 3: parent_2 = gene_group[2]
    elif parent_2 == 4: parent_2 = gene_group[3]
    elif parent_2 == 5: parent_2 = gene_group[4]
    '''
    return gene_group[p1], gene_group[p2], gene_group[p3], gene_group[p4], gene_group[p5], gene_group[p6], gene_group[p7], gene_group[p8], gene_group[p9], gene_group[p10], gene_group[p11], gene_group[p12], gene_group[p13], gene_group[p14], gene_group[p15], gene_group[p16], gene_group[p17], gene_group[p18], gene_group[p19], gene_group[p20], gene_group[p21], gene_group[p22], gene_group[p23], gene_group[p24], gene_group[p25], gene_group[p26], gene_group[p27], gene_group[p28], gene_group[p29], gene_group[p30]

CrossOver

In [11]:
def CrossOver(parent_gene_1, parent_gene_2):
    learning_rate_1 = parent_gene_1[:8]
    drop_out_1_1 = parent_gene_1[8:12]
    drop_out_2_1 = parent_gene_1[12:16]
    batch_size_1 = parent_gene_1[16:19]
    layer_1 = parent_gene_1[19:]
    
    learning_rate_2 = parent_gene_2[:8]
    drop_out_1_2 = parent_gene_2[8:12]
    drop_out_2_2 = parent_gene_2[12:16]
    batch_size_2 = parent_gene_2[16:19]
    layer_2 = parent_gene_2[19:]
    
    tmp = learning_rate_1[:4]
    tmp2 = learning_rate_1[4:]
    tmp3 = learning_rate_2[:4]
    tmp4 = learning_rate_2[4:]
    learning_rate_1_new = tmp + tmp4
    learning_rate_2_new = tmp3 + tmp2
    
    tmp = drop_out_1_1[:2]
    tmp2 = drop_out_1_1[2:]
    tmp3 = drop_out_1_2[:2]
    tmp4 = drop_out_1_2[2:]
    drop_out_1_1_new = tmp + tmp4
    drop_out_1_2_new = tmp3 + tmp2
    
    
    tmp = drop_out_2_1[:2]
    tmp2 = drop_out_2_1[2:]
    tmp3 = drop_out_2_2[:2]
    tmp4 = drop_out_2_2[2:]
    drop_out_2_1_new = tmp + tmp4
    drop_out_2_2_new = tmp3 + tmp2
    
    tmp = batch_size_1[:1]
    tmp2 = batch_size_1[1:]
    tmp3 = batch_size_2[:1]
    tmp4 = batch_size_2[1:]
    batch_size_1_new = tmp + tmp4
    batch_size_2_new = tmp3 + tmp2
    
    tmp = layer_1[:1]
    tmp2 = layer_1[1:]
    tmp3 = layer_2[:1]
    tmp4 = layer_2[1:]
    layer_1_new = tmp + tmp4
    layer_2_new = tmp3 + tmp2
    
    gene_1 = learning_rate_1_new + drop_out_1_1_new + drop_out_2_1_new + batch_size_1_new + layer_1_new
    gene_2 = learning_rate_2_new + drop_out_1_2_new + drop_out_2_2_new + batch_size_2_new + layer_2_new
    
    return gene_1, gene_2


Mutation 

In [12]:
def change(num):
    if num == 0: return 1
    elif num == 1: return 0

In [13]:
def Mutation(gene): #
    p_m = 0.05  #줄인다 0.05 아정도
    t = random.random()
    if p_m > t:
        gene[3] = change(gene[3]) # learning_rate의 4번쩨 원소
    
    t = random.random()
    if p_m > t:
        gene[9] = change(gene[9]) # drop_out_1의 첫번째 원소
    
    t = random.random()
    if p_m > t:
        gene[12] = change(gene[12]) #drop_out_2의 첫번째 원소
    
    t = random.random()
    if p_m > t:
        gene[13] = change(gene[13]) #batch_size의 두번째 원소
    
        
    t = random.random()
    if p_m > t:
        gene[20] = change(gene[20]) # layer의 마지막원소

    return gene

In [14]:
def make_new_population(cost, gene_group):
    tmp_dic = {0:cost[0], 1:cost[1], 2:cost[2], 3:cost[3], 4:cost[4], 5:cost[5], 6:cost[6], 7:cost[7], 8:cost[8], 9:cost[9],
              10:cost[10], 11:cost[11], 12:cost[12], 13:cost[13], 14:cost[14], 15:cost[15], 16:cost[16], 17:cost[17], 18:cost[18], 19:cost[19],
              20:cost[20], 21:cost[21], 22:cost[22], 23:cost[23], 24:cost[24], 25:cost[25], 26:cost[26], 27:cost[27], 28:cost[28], 29:cost[29],
              30:cost[30], 31:cost[31], 32:cost[32], 33:cost[33], 34:cost[34], 35:cost[35], 36:cost[36], 37:cost[37], 38:cost[38], 39:cost[39],
              40:cost[40], 41:cost[41], 42:cost[42], 43:cost[43], 44:cost[44], 45:cost[45], 46:cost[46], 47:cost[47], 48:cost[48], 49:cost[49]}
    cost_tmp = sorted(cost, reverse=True) #cost값은 변하지 않음
    
    for keys, values in tmp_dic.items():
        if cost_tmp[0] == values:
            idx_1 = keys
        if cost_tmp[1] == values:
            idx_2 = keys
        if cost_tmp[2] == values:
            idx_3 = keys
        if cost_tmp[3] == values:
            idx_4 = keys        
        if cost_tmp[4] == values:
            idx_5 = keys    
        if cost_tmp[5] == values:
            idx_6 = keys
        if cost_tmp[6] == values:
            idx_7 = keys
        if cost_tmp[7] == values:
            idx_8 = keys
        if cost_tmp[8] == values:
            idx_9 = keys
        if cost_tmp[9] == values:
            idx_10 = keys
            
        if cost_tmp[10] == values:
            idx_11 = keys
        if cost_tmp[11] == values:
            idx_12 = keys
        if cost_tmp[12] == values:
            idx_13 = keys
        if cost_tmp[13] == values:
            idx_14 = keys
        if cost_tmp[14] == values:
            idx_15 = keys
        if cost_tmp[15] == values:
            idx_16 = keys
        if cost_tmp[16] == values:
            idx_17 = keys
        if cost_tmp[17] == values:
            idx_18 = keys
        if cost_tmp[18] == values:
            idx_19 = keys
        if cost_tmp[19] == values:
            idx_20 = keys


    p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12, p13, p14, p15, p16, p17, p18, p19, p20, p21, p22, p23, p24, p25, p26, p27, p28, p29, p30 = Selection(cost, gene_group)

    print("p1: ", p1)
    child_gene_1, child_gene_2 = CrossOver(p1, p2)
    print("cross: ", child_gene_1)
    child_gene_1 = Mutation(child_gene_1)
    child_gene_2 = Mutation(child_gene_2)
    print("mu: ", child_gene_1)
    
    print("p3: ", p3)
    child_gene_3, child_gene_4 = CrossOver(p3, p4)
    print("cross: ", child_gene_3)
    child_gene_3 = Mutation(child_gene_3)
    child_gene_4 = Mutation(child_gene_4)
    print("mu: ", child_gene_3)
    
    child_gene_5, child_gene_6 = CrossOver(p5, p6)
    child_gene_5 = Mutation(child_gene_5)
    child_gene_6 = Mutation(child_gene_6)
    
    child_gene_7, child_gene_8 = CrossOver(p7, p8)
    child_gene_7 = Mutation(child_gene_7)
    child_gene_8 = Mutation(child_gene_8)
    
    child_gene_9, child_gene_10 = CrossOver(p9, p10)
    child_gene_9 = Mutation(child_gene_9)
    child_gene_10 = Mutation(child_gene_10)
    
    child_gene_11, child_gene_12 = CrossOver(p11, p12)
    child_gene_11 = Mutation(child_gene_11)
    child_gene_12 = Mutation(child_gene_12)
    
    child_gene_13, child_gene_14 = CrossOver(p13, p14)
    child_gene_13 = Mutation(child_gene_13)
    child_gene_14 = Mutation(child_gene_14)
    
    child_gene_15, child_gene_16 = CrossOver(p15, p16)
    child_gene_15 = Mutation(child_gene_15)
    child_gene_16 = Mutation(child_gene_16)
    
    child_gene_17, child_gene_18 = CrossOver(p17, p18)
    child_gene_17 = Mutation(child_gene_17)
    child_gene_18 = Mutation(child_gene_18)
    
    child_gene_19, child_gene_20 = CrossOver(p19, p20)
    child_gene_19 = Mutation(child_gene_19)
    child_gene_20 = Mutation(child_gene_20)
    
    child_gene_21, child_gene_22 = CrossOver(p21, p22)
    child_gene_21 = Mutation(child_gene_21)
    child_gene_22 = Mutation(child_gene_22)
    
    child_gene_23, child_gene_24 = CrossOver(p23, p24)
    child_gene_23 = Mutation(child_gene_23)
    child_gene_24 = Mutation(child_gene_24)
    
    child_gene_25, child_gene_26 = CrossOver(p25, p26)
    child_gene_25 = Mutation(child_gene_25)
    child_gene_26 = Mutation(child_gene_26)
    
    child_gene_27, child_gene_28 = CrossOver(p27, p28)
    child_gene_27 = Mutation(child_gene_27)
    child_gene_28 = Mutation(child_gene_28)
    
    child_gene_29, child_gene_30 = CrossOver(p29, p30)
    child_gene_29 = Mutation(child_gene_29)
    child_gene_30 = Mutation(child_gene_30)

    idx = [idx_1, idx_2, idx_3, idx_4, idx_5, idx_6, idx_7, idx_8, idx_9, idx_10,
          idx_11, idx_12, idx_13, idx_14, idx_15, idx_16, idx_17, idx_18, idx_19, idx_20]
   
    new_population = [gene_group[idx[0]],gene_group[idx[1]], gene_group[idx[2]], gene_group[idx[3]],gene_group[idx[4]], 
                      gene_group[idx[5]],gene_group[idx[6]],gene_group[idx[7]],gene_group[idx[8]],gene_group[idx[9]],
                      gene_group[idx[10]], gene_group[idx[11]], gene_group[idx[12]],gene_group[idx[13]], gene_group[idx[14]],
                      gene_group[idx[15]], gene_group[idx[16]],gene_group[idx[17]],gene_group[idx[18]], gene_group[idx[19]],
                      child_gene_1,child_gene_2,child_gene_3,child_gene_4,child_gene_5,child_gene_6,child_gene_7,child_gene_8,
                      child_gene_9,child_gene_10,child_gene_11,child_gene_12,child_gene_13,child_gene_14,child_gene_15,
                      child_gene_16,child_gene_17,child_gene_18,child_gene_19,child_gene_20,child_gene_21,child_gene_22,child_gene_23,
                      child_gene_24,child_gene_25,child_gene_26,child_gene_27,child_gene_28,child_gene_29,child_gene_30]
    
    return new_population, idx

In [15]:
def save_data(cost, gene_group, cost_max, lr_max, do1_max, do2_max, bat_max, lay_max):
    a = max(cost)
    #print("max cost : ", a)
    b = cost.index(max(cost))
    gene = gene_group[b] 
    cost_max.append(float(a))
    print("cost_max =  ", cost_max)
    
    global learning_rate_dic
    print("max gene : ", gene)
    learning_rate = gene[:8]
    learning_rate = ''.join(map(str, learning_rate))
    for key, value in learning_rate_dic.items():
        if value == learning_rate:
            learning_rate = key
            break
    lr_max.append(float(learning_rate))
    print("lr_max = ", lr_max)
    
    global drop_out_dic
    drop_out_1 = gene[8:12]
    drop_out_1 = ''.join(map(str, drop_out_1))
    for key, value in drop_out_dic.items():
        if value == drop_out_1:
            drop_out_1 = key
            break
    do1_max.append(float(drop_out_1))
    print("do1_max = ", do1_max)
    
    drop_out_2 = gene[12:16]
    drop_out_2 = ''.join(map(str, drop_out_2))
    for key, value in drop_out_dic.items():
        if value == drop_out_2:
            drop_out_2 = key
            break
    do2_max.append(float(drop_out_2))
    print("do2_max = ", do2_max)
    
    global batch_size_dic
    batch_size = gene[16:19]
    batch_size = ''.join(map(str, batch_size))
    for key, value in batch_size_dic.items():
        if value == batch_size:
            batch_size = key
            break
    bat_max.append(float(batch_size))
    print("bat_max = ", bat_max)
    
    global layer_dic
    layer = gene[19:]
    layer = ''.join(map(str, layer))
    for key, value in layer_dic.items():
        if value == layer:
            layer = key
            break
    lay_max.append(float(layer))
    print("lay_max =  ", lay_max)

    return cost_max, lr_max, do1_max, do2_max, bat_max, lay_max


실행부분

- 초기값 확인

In [16]:
init()

In [17]:

#Genetic Algorithm 실행부분

gene_1 = Initialize()
gene_1_binary = Encoding(gene_1) # 이진화로 한 비트씩 나오온다.

gene_2 = Initialize()
gene_2_binary = Encoding(gene_2) # 이진화로 한 비트씩 나오온다.

gene_3 = Initialize()
gene_3_binary = Encoding(gene_3) # 이진화로 한 비트씩 나오온다.

gene_4 = Initialize()
gene_4_binary = Encoding(gene_4) # 이진화로 한 비트씩 나오온다.

gene_5 = Initialize()
gene_5_binary = Encoding(gene_5) # 이진화로 한 비트씩 나오온다.

gene_6 = Initialize()
gene_6_binary = Encoding(gene_6) # 이진화로 한 비트씩 나오온다.


gene_7 = Initialize()
gene_7_binary = Encoding(gene_7) # 이진화로 한 비트씩 나오온다.

gene_8 = Initialize()
gene_8_binary = Encoding(gene_8) # 이진화로 한 비트씩 나오온다.


gene_9 = Initialize()
gene_9_binary = Encoding(gene_9) # 이진화로 한 비트씩 나오온다.

gene_10 = Initialize()
gene_10_binary = Encoding(gene_10) # 이진화로 한 비트씩 나오온다.

gene_11 = Initialize()
gene_11_binary = Encoding(gene_11) # 이진화로 한 비트씩 나오온다.

gene_12 = Initialize()
gene_12_binary = Encoding(gene_12) # 이진화로 한 비트씩 나오온다.

gene_13 = Initialize()
gene_13_binary = Encoding(gene_13) # 이진화로 한 비트씩 나오온다.

gene_14 = Initialize()
gene_14_binary = Encoding(gene_14) # 이진화로 한 비트씩 나오온다.

gene_15 = Initialize()
gene_15_binary = Encoding(gene_15) # 이진화로 한 비트씩 나오온다.

gene_16 = Initialize()
gene_16_binary = Encoding(gene_16) # 이진화로 한 비트씩 나오온다.

gene_17 = Initialize()
gene_17_binary = Encoding(gene_17) # 이진화로 한 비트씩 나오온다.

gene_18 = Initialize()
gene_18_binary = Encoding(gene_18) # 이진화로 한 비트씩 나오온다.

gene_19 = Initialize()
gene_19_binary = Encoding(gene_19) # 이진화로 한 비트씩 나오온다.

gene_20 = Initialize()
gene_20_binary = Encoding(gene_20) # 이진화로 한 비트씩 나오온다.

gene_21 = Initialize()
gene_21_binary = Encoding(gene_21) # 이진화로 한 비트씩 나오온다.

gene_22 = Initialize()
gene_22_binary = Encoding(gene_22) # 이진화로 한 비트씩 나오온다.

gene_23 = Initialize()
gene_23_binary = Encoding(gene_23) # 이진화로 한 비트씩 나오온다.

gene_24 = Initialize()
gene_24_binary = Encoding(gene_24) # 이진화로 한 비트씩 나오온다.

gene_25 = Initialize()
gene_25_binary = Encoding(gene_25) # 이진화로 한 비트씩 나오온다.

gene_26 = Initialize()
gene_26_binary = Encoding(gene_26) # 이진화로 한 비트씩 나오온다.

gene_27 = Initialize()
gene_27_binary = Encoding(gene_27) # 이진화로 한 비트씩 나오온다.

gene_28 = Initialize()
gene_28_binary = Encoding(gene_28) # 이진화로 한 비트씩 나오온다.

gene_29 = Initialize()
gene_29_binary = Encoding(gene_29) # 이진화로 한 비트씩 나오온다.

gene_30 = Initialize()
gene_30_binary = Encoding(gene_30) # 이진화로 한 비트씩 나오온다.

gene_31 = Initialize()
gene_31_binary = Encoding(gene_31) # 이진화로 한 비트씩 나오온다.

gene_32 = Initialize()
gene_32_binary = Encoding(gene_32) # 이진화로 한 비트씩 나오온다.

gene_33 = Initialize()
gene_33_binary = Encoding(gene_33) # 이진화로 한 비트씩 나오온다.

gene_34 = Initialize()
gene_34_binary = Encoding(gene_34) # 이진화로 한 비트씩 나오온다.

gene_35 = Initialize()
gene_35_binary = Encoding(gene_35) # 이진화로 한 비트씩 나오온다.

gene_36 = Initialize()
gene_36_binary = Encoding(gene_36) # 이진화로 한 비트씩 나오온다.

gene_37 = Initialize()
gene_37_binary = Encoding(gene_37) # 이진화로 한 비트씩 나오온다.

gene_38 = Initialize()
gene_38_binary = Encoding(gene_38) # 이진화로 한 비트씩 나오온다.

gene_39 = Initialize()
gene_39_binary = Encoding(gene_39) # 이진화로 한 비트씩 나오온다.

gene_40 = Initialize()
gene_40_binary = Encoding(gene_40) # 이진화로 한 비트씩 나오온다.

gene_41 = Initialize()
gene_41_binary = Encoding(gene_41) # 이진화로 한 비트씩 나오온다.

gene_42 = Initialize()
gene_42_binary = Encoding(gene_42) # 이진화로 한 비트씩 나오온다.

gene_43 = Initialize()
gene_43_binary = Encoding(gene_43) # 이진화로 한 비트씩 나오온다.

gene_44 = Initialize()
gene_44_binary = Encoding(gene_44) # 이진화로 한 비트씩 나오온다.

gene_45 = Initialize()
gene_45_binary = Encoding(gene_45) # 이진화로 한 비트씩 나오온다.

gene_46 = Initialize()
gene_46_binary = Encoding(gene_46) # 이진화로 한 비트씩 나오온다.

gene_47 = Initialize()
gene_47_binary = Encoding(gene_47) # 이진화로 한 비트씩 나오온다.

gene_48 = Initialize()
gene_48_binary = Encoding(gene_48) # 이진화로 한 비트씩 나오온다.

gene_49 = Initialize()
gene_49_binary = Encoding(gene_49) # 이진화로 한 비트씩 나오온다.

gene_50 = Initialize()
gene_50_binary = Encoding(gene_50) # 이진화로 한 비트씩 나오온다.


In [18]:
gene_group = [gene_1_binary, gene_2_binary, gene_3_binary, gene_4_binary, gene_5_binary,
             gene_6_binary, gene_7_binary, gene_8_binary, gene_9_binary,gene_10_binary,
             gene_11_binary, gene_12_binary, gene_13_binary, gene_14_binary, gene_15_binary,
             gene_16_binary, gene_17_binary, gene_18_binary, gene_19_binary,gene_20_binary,
             gene_21_binary, gene_22_binary, gene_23_binary, gene_24_binary, gene_25_binary,
             gene_26_binary, gene_27_binary, gene_28_binary, gene_29_binary,gene_30_binary,
             gene_31_binary, gene_32_binary, gene_33_binary, gene_34_binary, gene_35_binary,
             gene_36_binary, gene_37_binary, gene_38_binary, gene_39_binary,gene_40_binary,
             gene_41_binary, gene_42_binary, gene_43_binary, gene_44_binary, gene_45_binary,
             gene_46_binary, gene_47_binary, gene_48_binary, gene_49_binary,gene_50_binary]
cost=[]
print(gene_group)

[[1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0], [1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1], [1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1], [0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0], [0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0], [1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1], [0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0], [1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1], [0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1], [0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0], [1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0], [0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0], [0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0], [0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1], [1, 1, 0, 1, 1, 0, 0, 0,

In [19]:

cost.append(Evaluation(gene_1_binary))
print("1 :", cost)

cost.append(Evaluation(gene_2_binary))
print("2 :", cost)

cost.append(Evaluation(gene_3_binary))
print("3 :", cost)

cost.append(Evaluation(gene_4_binary))
print("4 :", cost)

cost.append(Evaluation(gene_5_binary))
print("5 :", cost)

cost.append(Evaluation(gene_6_binary))
print("6 :", cost)

cost.append(Evaluation(gene_7_binary))
print("7 :", cost)

cost.append(Evaluation(gene_8_binary))
print("8 :", cost)

cost.append(Evaluation(gene_9_binary))
print("9 :", cost)

cost.append(Evaluation(gene_10_binary))
print("10 :", cost)


cost.append(Evaluation(gene_11_binary))
print("11 :", cost)

cost.append(Evaluation(gene_12_binary))
print("12 :", cost)

cost.append(Evaluation(gene_13_binary))
print("13 :", cost)

cost.append(Evaluation(gene_14_binary))
print("14 :", cost)

cost.append(Evaluation(gene_15_binary))
print("15 :", cost)

cost.append(Evaluation(gene_16_binary))
print("16 :", cost)

cost.append(Evaluation(gene_17_binary))
print("17 :", cost)

cost.append(Evaluation(gene_18_binary))
print("18 :", cost)

cost.append(Evaluation(gene_19_binary))
print("19 :", cost)

cost.append(Evaluation(gene_20_binary))
print("20 :", cost)


cost.append(Evaluation(gene_21_binary))
print("21 :", cost)

cost.append(Evaluation(gene_22_binary))
print("22 :", cost)

cost.append(Evaluation(gene_23_binary))
print("23 :", cost)

cost.append(Evaluation(gene_24_binary))
print("24 :", cost)

cost.append(Evaluation(gene_25_binary))
print("25 :", cost)

cost.append(Evaluation(gene_26_binary))
print("26 :", cost)

cost.append(Evaluation(gene_27_binary))
print("27 :", cost)

cost.append(Evaluation(gene_28_binary))
print("28 :", cost)

cost.append(Evaluation(gene_29_binary))
print("29 :", cost)

cost.append(Evaluation(gene_30_binary))
print("30 :", cost)


cost.append(Evaluation(gene_31_binary))
print("31 :", cost)

cost.append(Evaluation(gene_32_binary))
print("32 :", cost)

cost.append(Evaluation(gene_33_binary))
print("33 :", cost)

cost.append(Evaluation(gene_34_binary))
print("34 :", cost)

cost.append(Evaluation(gene_35_binary))
print("35 :", cost)

cost.append(Evaluation(gene_36_binary))
print("36 :", cost)

cost.append(Evaluation(gene_37_binary))
print("37 :", cost)

cost.append(Evaluation(gene_38_binary))
print("38 :", cost)

cost.append(Evaluation(gene_39_binary))
print("39 :", cost)

cost.append(Evaluation(gene_40_binary))
print("40 :", cost)

cost.append(Evaluation(gene_41_binary))
print("41 :", cost)

cost.append(Evaluation(gene_42_binary))
print("42 :", cost)

cost.append(Evaluation(gene_43_binary))
print("43 :", cost)

cost.append(Evaluation(gene_44_binary))
print("44 :", cost)

cost.append(Evaluation(gene_45_binary))
print("45 :", cost)

cost.append(Evaluation(gene_46_binary))
print("46 :", cost)

cost.append(Evaluation(gene_47_binary))
print("47 :", cost)

cost.append(Evaluation(gene_48_binary))
print("48 :", cost)

cost.append(Evaluation(gene_49_binary))
print("49 :", cost)

cost.append(Evaluation(gene_50_binary))
print("50 :", cost)


Evaluation 실행
Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Instructions for updating:
Use `argmax` instead
Instructions for updating:
Use `tf.global_variables_initializer` instead.
1 번째 
2 번째 
3 번째 
4 번째 
5 번째 
6 번째 
7 번째 
8 번째 
9 번째 
10 번째 
11 번째 
12 번째 
13 번째 
14 번째 
15 번째 
16 번째 
17 번째 
18 번째 
19 번째 
20 번째 
test accuracy :  0.098
1 : [0.098]
Evaluation 실행
Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
1 번째 
2 번째 
3 번째 
4 번째 
5 번째 
6 번째 
7 번째 
8 번째 
9 번째 
10 번째 
11 번째 
12 번째 
13 번째 
14 번째 
15 번째 
16 번째 
17 번

Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
1 번째 
2 번째 
3 번째 
4 번째 
5 번째 
6 번째 
7 번째 
8 번째 
9 번째 
10 번째 
11 번째 
12 번째 
13 번째 
14 번째 
15 번째 
16 번째 
17 번째 
18 번째 
19 번째 
20 번째 
test accuracy :  0.098
18 : [0.098, 0.098, 0.098, 0.098, 0.098, 0.098, 0.99, 0.098, 0.9726, 0.098, 0.9876, 0.9759, 0.098, 0.098, 0.098, 0.098, 0.098, 0.098]
Evaluation 실행
Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
1 번째 
2 번째 
3 번째 
4 번째 
5 번째 
6 번째 
7 번째 
8 번째 
9 번째 
10 번째 
11 번째 
12 번째 
13 번째 
14 번째 
15 번째 
16 번째 
17 번째 
18 번째 
19 번째 
20 번째 
test accuracy :  0.9381
19 : [0.098, 0.098, 0.098, 0.098, 0.098, 0.098, 0.99, 0.098, 0.9726, 0.098, 0.9876, 0.9759, 0.098, 0.098, 0.098, 0.098, 0.098, 0.098, 0.9381]
Evaluation 실행
Extracting MNIST_data/train-images-idx3-ubyte.gz

Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
1 번째 
2 번째 
3 번째 
4 번째 
5 번째 
6 번째 
7 번째 
8 번째 
9 번째 
10 번째 
11 번째 
12 번째 
13 번째 
14 번째 
15 번째 
16 번째 
17 번째 
18 번째 
19 번째 
20 번째 
test accuracy :  0.098
33 : [0.098, 0.098, 0.098, 0.098, 0.098, 0.098, 0.99, 0.098, 0.9726, 0.098, 0.9876, 0.9759, 0.098, 0.098, 0.098, 0.098, 0.098, 0.098, 0.9381, 0.098, 0.9669, 0.098, 0.098, 0.098, 0.9299, 0.098, 0.098, 0.9842, 0.098, 0.9366, 0.9678, 0.098, 0.098]
Evaluation 실행
Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
1 번째 
2 번째 
3 번째 
4 번째 
5 번째 
6 번째 
7 번째 
8 번째 
9 번째 
10 번째 
11 번째 
12 번째 
13 번째 
14 번째 
15 번째 
16 번째 
17 번째 
18 번째 
19 번째 
20 번째 
test accuracy :  0.098
34 : [0.098, 0.098, 0.098, 0.098, 0.098, 0.098, 0.99, 0.098, 0.9726, 0.098, 0.9876, 0.9759, 0.

Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
1 번째 
2 번째 
3 번째 
4 번째 
5 번째 
6 번째 
7 번째 
8 번째 
9 번째 
10 번째 
11 번째 
12 번째 
13 번째 
14 번째 
15 번째 
16 번째 
17 번째 
18 번째 
19 번째 
20 번째 
test accuracy :  0.9902
46 : [0.098, 0.098, 0.098, 0.098, 0.098, 0.098, 0.99, 0.098, 0.9726, 0.098, 0.9876, 0.9759, 0.098, 0.098, 0.098, 0.098, 0.098, 0.098, 0.9381, 0.098, 0.9669, 0.098, 0.098, 0.098, 0.9299, 0.098, 0.098, 0.9842, 0.098, 0.9366, 0.9678, 0.098, 0.098, 0.098, 0.098, 0.9731, 0.098, 0.098, 0.098, 0.098, 0.098, 0.098, 0.098, 0.098, 0.098, 0.9902]
Evaluation 실행
Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
1 번째 
2 번째 
3 번째 
4 번째 
5 번째 
6 번째 
7 번째 
8 번째 
9 번째 
10 번째 
11 번째 
12 번째 
13 번째 
14 번째 
15 번째 
16 번째 
17 번째 
18 번째 
19 번째 
20 번째 
test accuracy :  0.9913

In [19]:
print("cost = ", cost)
print("gene_group = ", gene_group)


cost_max = [] 
lr_max = [] 
do1_max = [] 
do2_max = [] 
bat_max =[] 
lay_max = []

cost_max, lr_max, do1_max, do2_max, bat_max, lay_max =  save_data(cost, gene_group, cost_max, lr_max, do1_max, do2_max, bat_max, lay_max)

cost =  []
gene_group =  [[1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0], [1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1], [1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1], [0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0], [0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0], [1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1], [0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0], [1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1], [0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1], [0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0], [1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0], [0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0], [0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0], [0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1],

ValueError: max() arg is an empty sequence